### Imports

In [1]:
import os
import pandas as pd
import numpy as np

### Parameters

In [2]:
num_samples = 50 # quantidade de exemplos

### Inputs

In [3]:
def generate_realistic_edge(is_good_link):
    if is_good_link:
        rssi_mean = -55
        rssi_std = 8
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -85, -30)
        
        etx = 1 + np.random.exponential(0.8)
        etx = np.clip(etx, 1, 8)
        
        delay = np.random.lognormal(2.3, 0.6)
        delay = np.clip(delay, 1, 80)
        
        busy = np.random.beta(2, 5)
        busy = np.clip(busy, 0.05, 0.7)
        
    else:
        rssi_mean = -75
        rssi_std = 10
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -95, -45)
        
        etx = 1 + np.random.gamma(3, 1.2)
        etx = np.clip(etx, 1.5, 15)
        
        delay = np.random.lognormal(3.5, 0.8)
        delay = np.clip(delay, 10, 150)
        
        busy = np.random.beta(5, 2)
        busy = np.clip(busy, 0.1, 1.0)
    
    rssi += np.random.uniform(-0.05, 0.05)
    etx += np.random.uniform(-0.01, 0.01)
    delay += np.random.uniform(-0.2, 0.2)
    busy += np.random.uniform(-0.005, 0.005)
    
    rssi = np.clip(rssi, -95, -30)
    etx = np.clip(etx, 1.0, 15)
    delay = np.clip(delay, 1, 150)
    busy = np.clip(busy, 0.0, 1.0)
    
    if rssi < -70:
        etx *= np.random.uniform(1.1, 1.4)
        delay *= np.random.uniform(1.05, 1.25)
        etx = min(etx, 15)
    
    if busy > 0.6:
        delay *= np.random.uniform(1.2, 1.6)
        etx *= np.random.uniform(1.05, 1.2)
        delay = min(delay, 150)
        etx = min(etx, 15)
    
    if etx > 7:
        delay *= np.random.uniform(1.1, 1.3)
        delay = min(delay, 150)
    
    rssi = round(rssi, 2)
    etx = round(etx, 3)
    delay = round(delay, 2)
    busy = round(busy, 4)
    
    return rssi, etx, delay, busy

edge_inputs = []
edge_labels = []
label_0_count = 0
label_1_count = 0
target_per_label = num_samples // 2

while len(edge_inputs) < num_samples:
    if label_0_count >= target_per_label:
        target_label = 1
    elif label_1_count >= target_per_label:
        target_label = 0
    else:
        target_label = 1 if np.random.random() < 0.52 else 0
    
    is_good = (target_label == 1)
    
    rssi, etx, delay, busy = generate_realistic_edge(is_good)
    
    if target_label == 0:
        label_0_count += 1
    else:
        label_1_count += 1
    
    edge_inputs.append([rssi, etx, delay, busy])
    edge_labels.append(target_label)

In [4]:
def generate_realistic_radnet_metrics(is_good_performance):
    if is_good_performance:
        efsr_edge = np.random.beta(9, 2)
        efsr_edge = 0.70 + (efsr_edge * 0.25)
        
    else:
        efsr_edge = np.random.beta(3, 6)
        efsr_edge = 0.25 + (efsr_edge * 0.35)
    
    efsr_edge += np.random.uniform(-0.02, 0.02)
    
    efsr_edge = np.clip(efsr_edge, 0.20, 0.98)
    
    efsr_edge = round(efsr_edge, 6)
    
    return efsr_edge

radnet_inputs = []
radnet_labels = []
good_performance_count = 0
poor_performance_count = 0
target_per_performance = num_samples // 2

while len(radnet_inputs) < num_samples:
    if good_performance_count >= target_per_performance:
        target_performance = 0
    elif poor_performance_count >= target_per_performance:
        target_performance = 1
    else:
        target_performance = 1 if np.random.random() < 0.52 else 0
    
    is_good = (target_performance == 1)
    efsr = generate_realistic_radnet_metrics(is_good)
    
    if target_performance == 0:
        poor_performance_count += 1
    else:
        good_performance_count += 1
    
    radnet_inputs.append([efsr])
    radnet_labels.append(target_performance)

### Save

In [5]:
edges_input = pd.DataFrame(edge_inputs, columns=["rssi", "etx", "delay", "busy_fraction"])
edges_input['label'] = edge_labels

os.makedirs("../inputs", exist_ok=True)
edges_input.to_csv("../inputs/edges_input.csv", index=False)

print("Edges Inputs salvo em ../inputs/edges_input.csv")

Edges Inputs salvo em ../inputs/edges_input.csv


In [6]:
radnet_input = pd.DataFrame(radnet_inputs, columns=["efsr_edge"])
radnet_input['label'] = radnet_labels

os.makedirs("../inputs", exist_ok=True)
radnet_input.to_csv("../inputs/radnet_input.csv", index=False)

print("Radnet Input salvo em ../inputs/radnet_input.csv")

Radnet Input salvo em ../inputs/radnet_input.csv
